In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [5]:
from pyspark.ml import PipelineModel

In [6]:
globals()['models_loaded'] = True
globals()['my_model'] = PipelineModel.load('models/channel_classifier')

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
            
    #predicting using custom model
    df_result = globals()['my_model'].transform(df)
    
    df_result.select("message","username","channel","predicted_label").show()

In [7]:
ssc = StreamingContext(sc, 10)

In [8]:
lines = ssc.socketTextStream("localhost", 8080)
lines.foreachRDD(process)

In [9]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [11]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


22/05/25 05:03:05 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.SocketInputStream.socketRead(SocketInputStream.java:115)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:168)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:181)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:392)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:121)
	at org.a

========= 2022-05-25 05:03:10 =========
+--------+--------------------+--------------------+--------------+
| channel|            datetime|             message|      username|
+--------+--------------------+--------------------+--------------+
|#cdawgva|2022-05-25T03:02:...|YOU WERE JUST LIK...|patrick2tothe4|
|#cdawgva|2022-05-25T03:03:...|                KEKW|        jhinz6|
|#cdawgva|2022-05-25T03:03:...|                 !ad|   c_midknight|
|#cdawgva|2022-05-25T03:03:...|Download and play...|      nightbot|
|#cdawgva|2022-05-25T03:03:...|                COCC|    kalaydo007|
|#cdawgva|2022-05-25T03:03:...|is that metal gea...|  tk_silentass|
|#cdawgva|2022-05-25T03:03:...|                HEHE|     maria3142|
|#cdawgva|2022-05-25T03:03:...|trying to make hi...|    theblu3jay|
|#cdawgva|2022-05-25T03:03:...|                 YEP|        dmoon0|
|#cdawgva|2022-05-25T03:03:...|                 YEP|      jhinpor4|
|#cdawgva|2022-05-25T03:03:...|no aims just enjo...|      cain_145|
|#cdawgv